In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import ast
import collections
from wordcloud import WordCloud, STOPWORDS
import json

In [ ]:
#### #### read val labels

with open('label_dictionary.json','r') as fp:
    label_dictionary_old = json.load(fp)
    
print(label_dictionary_old)

num_selected = 1000

val_labels = np.load('random_pairs_2k/val_labels_processed.npy')[:num_selected]

random_subembeddings = np.load('random_pairs_2k/cumulative_sub_embedding_output_1_random.npy')[:num_selected,:]
#random_embeddings = random_embeddings+np.random.randn(*random_embeddings.shape)/10000


# Compute 90th Percentile Thresholds for Sub-Embeddings Samples

In [2]:
# get embedding percentile thresholds

subembeddings_for_percentile_comp= np.squeeze(np.load('alexnet_imgnet_sub_embeddings_prelu_noise.npy'))
subembeddings_for_percentile_comp = subembeddings_for_percentile_comp + np.random.randn(*subembeddings_for_percentile_comp.shape)/10000
sorted_indices_subembedding = np.argsort(subembeddings_for_percentile_comp,axis = 0)
indices_90_subembedding = sorted_indices_subembedding[900,:]
subembedding_threshold_90 = subembeddings_for_percentile_comp[indices_90_subembedding, range(4096)].reshape(1,4096)

# Compute Number of Co-active Features Between Sub-Embeddings of All Samples

In [3]:
random_subemb_active = (random_subembeddings >= subembedding_threshold_90)*1
random_subemb_active_t = random_subemb_active.T
random_subemb_active_expanded = np.expand_dims(random_subemb_active, axis=2)
random_subemb_active_t_expanded = np.expand_dims(random_subemb_active_t, axis=0)
co_active_feats = random_subemb_active_expanded*random_subemb_active_t_expanded
co_active_features_random_subemb = np.matmul(random_subemb_active,random_subemb_active_t)
co_active_features_random_subemb_sorted = np.argsort(-co_active_features_random_subemb,axis = 1)
average_random_subemb = np.mean(co_active_features_random_subemb)


NameError: name 'random_subembeddings' is not defined

# Compute and Plot Histogram of Number of Co-Active Features Between All Pairs of Sub-Embeddings (sanity-check)

In [ ]:
bins=np.linspace(np.min(co_active_features_random_subemb), np.max(co_active_features_random_subemb),50)
plt.figure(figsize=(7,7))
plt.title('histograms of number of co-active features in the sub-embedding \n average number of co-active features (sub-embedding layer) = '+str(average_random_subemb), fontsize=14)
plt.hist(co_active_features_random_subemb.flatten(),bins=bins, label = 'subembedding', alpha = 0.5)
plt.ylabel('count',fontsize=14)
plt.xlabel('number of co-active features',fontsize=14)
plt.legend()
plt.show()
plt.close()

In [ ]:
label_dictionary = {}
for key in label_dictionary_old:
    label_dictionary[float(key)] = label_dictionary_old[key]

# Compute Number of Co-Active Pairs Resulting from Same and Different Classes Across All-Features

In [ ]:
feats_same = np.zeros(4096)
feats_diff = np.zeros(4096)

feats_num_unique_classes_same = np.zeros(4096)
#feats_unique_classes_same = {}

feats_num_unique_classes_diff = np.zeros(4096)
#feats_unique_classes_diff = {}

feature_cloud = collections.defaultdict(str)

for feature in range(4096):
    feats_same_classes = []
    feats_diff_classes = []
    
    # get sum of activations for a features
    co_active_sum_of_activations_sorted_args = np.load('co_active_sum_of_sub_activations_sorted_args_'+str(feature)+'.npy')
    
    for ind in range(0,len(co_active_sum_of_activations_sorted_args),2):
        x = int(co_active_sum_of_activations_sorted_args[ind]/num_selected)
        y = co_active_sum_of_activations_sorted_args[ind]%num_selected
        
        
        if co_active_feats[x,feature,y] == 0:
            break
            
        if x == y:
            continue
        else:
            class_x = val_labels[x]
            class_y = val_labels[y]
            
            label_x = label_dictionary[class_x]
            label_y = label_dictionary[class_y]
            
            feature_cloud[feature] = feature_cloud[feature]+' '+label_x+' '+label_y
            if class_x != class_y:
                feats_diff[feature] += 1
                if class_x not in feats_diff_classes:
                    feats_diff_classes.append(class_x)
                if class_y not in feats_diff_classes:
                    feats_diff_classes.append(class_y)
            else:
                feats_same[feature] += 1
                if class_x not in feats_same_classes:
                    feats_same_classes.append(class_x)
                
    feats_num_unique_classes_same[feature] = len(feats_same_classes)
    #feats_unique_classes_same[feature] = feats_same_classes
    feats_num_unique_classes_diff[feature] = len(feats_diff_classes)
    #feats_unique_classes_diff[feature] = feats_diff_classes
    

# Histogram of Number of Co-Active Pairs Resulting from Same and Different Classes Across All Features

In [ ]:
bins=np.linspace(np.min([np.min(feats_same),np.min(feats_diff)]),np.max([np.max(feats_same),np.max(feats_diff)]),100)
plt.figure(figsize=(10,20))
fig, axs = plt.subplots(1,2, figsize=(20,10))
axs[0].set_title('histograms of number of co-activations for \n image-pairs from same class \n ', fontsize=14)
axs[0].hist(feats_same,bins=15, label = 'co-activation from same-class')
axs[0].set_ylabel('count',fontsize=14)
axs[0].set_xlabel('number of co-active sub-embedding features',fontsize=14)
axs[0].legend()
axs[1].set_title('histograms of number of co-activations for \n image-pairs from different class', fontsize=14)
axs[1].hist(feats_diff,bins=15, label = 'co-activation from different-class', color='orange')
axs[1].set_ylabel('count',fontsize=14)
axs[1].set_xlabel('number of co-active sub-embedding features',fontsize=14)
axs[1].legend()
plt.ylabel('count',fontsize=14)
plt.xlabel('number of co-active sub-embedding features',fontsize=14)
plt.legend()
plt.show()
plt.close()

# Histogram of Number Unique Classes Across Image-Pairs With the Same Class Label and Across Image-Pairs With the Different Class Label

In [ ]:
bins=np.linspace(np.min([np.min(feats_same),np.min(feats_diff)]),np.max([np.max(feats_same),np.max(feats_diff)]),100)
fig, axs = plt.subplots(1,2, figsize=(20,10))
axs[0].set_title('histograms of number of unique classes per co-active feature \n for image-pairs with the same class label', fontsize=14)
axs[0].hist(feats_num_unique_classes_same,bins=10, label = 'co-activation from different-class')
axs[0].set_ylabel('count',fontsize=14)
axs[0].set_xlabel('number of unique classes per co-active feature',fontsize=14)
axs[0].legend()

print(max(feats_num_unique_classes_diff))
axs[1].set_title('histograms of number of unique classes per co-active feature \n for image-pairs with the different class labels', fontsize=14)
axs[1].hist(feats_num_unique_classes_diff,bins=10, label = 'co-activation from different-class', color='orange')
axs[1].set_ylabel('count',fontsize=14)
axs[1].set_xlabel('number of unique classes per co-active feature',fontsize=14)
axs[1].legend()

# Display Top-10 Pairs (With Highest Sum of Feature Values) for the Purest Co-Active Features

In [ ]:
print(np.max(feats_same))
print(np.min(feats_same))
# get max_co_active_features

sorted_feats_same = np.argsort(-feats_same.flatten())[:10]


print('Image Pairs With Highest Sum of Sub-Embedding Co-Activation Values for Most Homogenous Features')


for feature in sorted_feats_same:
    # get sum of activations for a features
    print('feature ',feature)
    print('number of co-active pairs from same class = ',feats_same[feature])
    print('number of co-active pairs from diff class = ',feats_diff[feature])
    
    #plot the feature cloud
    
    wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='salmon', colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(feature_cloud[feature])
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.title('Word Cloud Made Using Class Labels from Co-Active Pairs')
    co_active_sum_of_activations_sorted_args = np.load('co_active_sum_of_sub_activations_sorted_args_'+str(feature)+'.npy')
    list_of_sorted_indices = []
    for ind in range(len(co_active_sum_of_activations_sorted_args)):
        x = int(co_active_sum_of_activations_sorted_args[ind]/num_selected)
        y = co_active_sum_of_activations_sorted_args[ind]%num_selected
        list_of_sorted_indices.append((x,y))
           
    # display pairs with highest activations
    pair_count = 1
    for ind in range(0,10,2):
        x,y = list_of_sorted_indices[ind][0], list_of_sorted_indices[ind][1]
        if co_active_feats[x,feature,y] == 0:
            break
        if x != y:
            class_x = val_labels[x]
            class_y = val_labels[y]
            label_x = label_dictionary[class_x]
            label_y = label_dictionary[class_y]
            fig,axs = plt.subplots(1,2, figsize = (8,4))
            print('Pair:',pair_count)
            pair_count += 1
            fig.tight_layout(pad=2)
            plt.subplots_adjust(top=1.0)
            img = mpimg.imread('random_pairs_cumulative/count_'+str(x)+'.jpg')
            axs[0].imshow(img, aspect="auto")
            axs[0].axis('off')
            axs[0].set_title('class: '+label_x,fontsize=10)
            img = mpimg.imread('random_pairs_cumulative/count_'+str(y)+'.jpg')
            axs[1].imshow(img, aspect="auto")
            axs[1].axis('off')
            axs[1].set_title('class: '+label_y,fontsize=10)
            plt.show()
            plt.close()

# Display Top-10 Pairs (With Highest Sum of Feature Values) for the Least-Pure Co-Active Features

In [ ]:
# get min_co_active_features
sorted_feats_diff = np.argsort(-feats_diff.flatten())[:10]

print('Image Pairs With Highest Sum of Sub-Embedding Co-Activation Values for Least Homogenous Features')

for feature in sorted_feats_diff:
    print('\n\nfeature: ',feature)
    print('num co-activations from different class :',feats_diff[feature])
    print('num co-activations from the same class :',feats_same[feature],'\n\n')
    
    wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='salmon', colormap='Pastel1', collocations=False, stopwords = STOPWORDS).generate(feature_cloud[feature])
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.title('Word Cloud Made Using Class Labels from Co-Active Pairs')
    co_active_sum_of_activations_sorted_args = np.load('co_active_sum_of_sub_activations_sorted_args_'+str(feature)+'.npy')
    
    co_active_current_feat = co_active_feats[:,feature,:]
    current_activations_all_activations = random_subembeddings[:,feature].reshape(1,-1)
    current_activations_all_activations_t = current_activations_all_activations.T
    sum_of_activations = current_activations_all_activations+current_activations_all_activations_t
    co_active_sum_of_activations = np.argsort(-(sum_of_activations*co_active_current_feat).flatten())
    list_of_sorted_indices = []
    for ind in range(len(co_active_sum_of_activations)):
        x = int(co_active_sum_of_activations[ind]/num_selected)
        y = co_active_sum_of_activations[ind]%num_selected
        list_of_sorted_indices.append((x,y))
           
    # display pairs with highest activations
    for ind in range(0,10,2):
        x,y = list_of_sorted_indices[ind][0], list_of_sorted_indices[ind][1]
        if x != y:
            class_x = val_labels[x]
            class_y = val_labels[y]
            label_x = label_dictionary[class_x]
            label_y = label_dictionary[class_y]
            print('Pair:',pair_count)
            pair_count += 1
            fig,axs = plt.subplots(1,2, figsize = (8,4))
            fig.tight_layout(pad=2)
            plt.subplots_adjust(top=1.0)
            img = mpimg.imread('random_pairs_cumulative/count_'+str(x)+'.jpg')
            axs[0].imshow(img, aspect="auto")
            axs[0].axis('off')
            axs[0].set_title('class: '+str(class_x),fontsize=10)
            img = mpimg.imread('random_pairs_cumulative/count_'+str(y)+'.jpg')
            axs[1].imshow(img, aspect="auto")
            axs[1].axis('off')
            axs[1].set_title('class: '+str(class_y),fontsize=10)
            plt.show()
            plt.close()